# Entrainement de modèles génératifs répondant aux développeurs

L'objectif de ce notebok est d'utiliser un modèle déjà entrainé et, grâce à un jeu de données de feedbacks sur du code, le spécialiser afin qu'il piusse répondre à des questions de développeurs.

## Download requirements

First install a library necessary to run quantized models.

In [ ]:
import nbformat

def fix_widget_metadata(path):
    # Load notebook
    nb = nbformat.read(path, as_version=nbformat.NO_CONVERT)

    # Check for widgets metadata
    if 'widgets' in nb['metadata']:
        widgets = nb['metadata']['widgets']
        # Ensure 'state' key exists
        if 'state' not in widgets:
            widgets['state'] = {}
        nb['metadata']['widgets'] = widgets
    else:
        # Optionally remove the key entirely
        nb['metadata'].pop('widgets', None)

    # Save fixed notebook
    nbformat.write(nb, path)
    print(f"✅ Fixed and saved: {path}")

# Usage example:
fix_widget_metadata("your_notebook.ipynb")


In [ ]:
pip install -U accelerate==1.8.1 peft==0.15.0 bitsandbytes==0.46.0 transformers==4.52.4 datasets

## Load a generative model

We are going to load a generative model, a well-known open model called **EleutherAI/gpt-neo-1.3B**

In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer
import torch

model_id = "EleutherAI/gpt-neo-1.3B"
#On pourra aussi essayer :
# Salesforce/codegen-350M-multi
# EleutherAI/gpt-neo-1.3B

from transformers import BitsAndBytesConfig


bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype="float16",
    bnb_4bit_use_double_quant=False,
)

tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModelForCausalLM.from_pretrained(model_id, quantization_config=bnb_config, device_map="auto")



In [ ]:
print(model)
# modèle de base pré entrainé pour prédiction next token



🚧 **TODO** 🚧

Experiment with the generation of the model.

In [ ]:
text = "Explain NullPointerException error"
DEVICE= "cuda:0"


inputs = tokenizer(text, return_tensors="pt").to(DEVICE)
outputs = model.generate(**inputs, max_new_tokens=200, temperature=1.0, do_sample=True)
print(tokenizer.decode(outputs[0]))

On constate ici que le modèle génère bien une réponse mais vraiment pas de manière pertinente.

Nous allons le fine tuné afin qu'il offre des réponses plus pertinenete et pouvant réellement apporté une réponse utile à un utilisateur expérimenté sur le sujet.

## Training

First load an instruction dataset.

In [ ]:
from datasets import load_dataset

# Chargement du dataset Code-Feedback
data = load_dataset("m-a-p/Code-Feedback")

# Explorer un exemple
print(data["train"][0])

In [ ]:
print(data["train"][0]["messages"][1]["content"])

In [ ]:
# Load tokenizer (change model name as needed)
tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")

def process_data(sample):
    #return tokenizer(sample["messages"][1][1]["content"], max_length=1000, truncation=True, padding="max_length")
    # batch["messages"] is a list of samples, so process each sample in the batch
    texts = [sample[1]["content"] for sample in sample["messages"]]
    tokenized = tokenizer(texts, max_length=1000, truncation=True, padding="max_length")
    return tokenized


# Map the function to the dataset in batched mode
data = data.map(process_data, batched=True)


In [ ]:
print(data["train"][0])

# system prompt : on apprend au modèle à répondre des questions
# les connaissances du modèle viennet de son entrainement, des données qu'on lui a fourni

Since this model is quite big, we are going to reduce the number of trainable parameters.

To do that, we use **Parameter Efficient Fine-Tuning** (PEFT).

In PEFT, we do not tune the full model. Only a small subsets of the parameters are trained, while the others are frozen (i.e., not updated during training).

🚧 **TODO** 🚧

Explain why PEFT reduces the memory cost.


In [ ]:
from peft import prepare_model_for_kbit_training

# This determines automatically which module can be used for quantized training inside the model
model = prepare_model_for_kbit_training(model)

In [ ]:
def print_trainable_parameters(model):
    """
    Prints the number of trainable parameters in the model.
    """
    trainable_params = 0
    all_param = 0
    for _, param in model.named_parameters():
        all_param += param.numel()
        if param.requires_grad:
            trainable_params += param.numel()
    print(
        f"trainable params: {trainable_params} || all params: {all_param} || trainable%: {100 * trainable_params / all_param}"
    )

The cell below adds a small amount of extra-paramter to the model using Lora technique, that is described here: https://arxiv.org/abs/2106.09685

In [ ]:
# r : ordre de grandeur de la pertubation que l'on va ajouter aux données


from peft import LoraConfig, get_peft_model

# Use default config
config = LoraConfig(
    r=8,
    lora_alpha=32,
    target_modules=["q_proj", "k_proj", "v_proj"],
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM"
)

model = get_peft_model(model, config)
print_trainable_parameters(model)

We now use the Trainer class of HuggingFace. On simple installations it can be very effective.

In [ ]:
import transformers

#tokenizer.pad_token = tokenizer.eos_token


tokenizer.add_special_tokens({'pad_token': '[PAD]'})
model.resize_token_embeddings(len(tokenizer))


# on garde la même dette
# le modèle sait déjà comment trouver le mot d'apres avec des mots du dico entier
# on lui apprend aussi à prédire le mot d'après mais avec des données propres


trainer = transformers.Trainer(
    model=model,
    train_dataset=data["train"],
    args=transformers.TrainingArguments(
        per_device_train_batch_size=16,
        gradient_accumulation_steps=1,
        max_steps=100,
        learning_rate=2e-4,
        fp16=True,
        logging_steps=1,
        output_dir="outputs",
        optim="paged_adamw_8bit",
        report_to="none"
    ),
    data_collator=transformers.DataCollatorForLanguageModeling(tokenizer, mlm=False)
)

model.config.use_cache = False
trainer.train()

In [ ]:
text = "### Instruction:\nPropose an outdoor activity.\n\n### Response:\n"
inputs = tokenizer(text, return_tensors="pt").to(DEVICE)
model.config.use_cache = True

outputs = model.generate(**inputs, max_new_tokens=200, temperature=1.0, do_sample=True)
print(tokenizer.decode(outputs[0]))

on a un modèle pas mal, en très peu de temps avec un milliard de paramètres, pas parfait mais réponds à ma question.

Pour améliorer :